In [ ]:
## Experiment to check how much monte carlo approach differs from the z  - 0.5 approx
import torch
import numpy as np 
from barrelnet.synthbarrel import random_cylinder, monte_carlo_volume_ratio
import matplotlib.pyplot as plt
from tqdm import tqdm


def sample_normals(num_samples, device='cuda'):
	"""
	Sample random unit vectors uniformly distributed on the upper hemisphere.
	
	Args:
		num_samples (int): Number of unit vectors to sample.
		device (str): Device to perform the computation ('cpu' or 'cuda').
		
	Returns:
		torch.Tensor: Tensor of shape (num_samples, 3) containing the sampled unit vectors.
	"""
	phi = torch.rand(num_samples, device=device) * 2 * torch.pi
	theta = torch.acos(torch.rand(num_samples, device=device))

	x = torch.sin(theta) * torch.cos(phi)
	y = torch.sin(theta) * torch.sin(phi)
	z = torch.cos(theta)

	return torch.stack((x, y, z), dim=1)


normals = sample_normals(10000).cpu().numpy()

height = 2.3 
radius = 1.0

gt, approx = [], []
for normal in tqdm(normals):
	shift = (np.random.rand(1) - 0.5)[0]
	origin = np.array([0., 0., shift*height])
	x1 = origin + normal*height/2
	x2 = origin - normal*height/2 

	a,b,c,d = 0,0,1,0 # ax + by + cz +d = 0 
	burial_frac_gt = 1 - monte_carlo_volume_ratio(100000, x1, x2, radius, a,b,c,d)
	burial_frac_approx = np.clip(0.5 - shift,0,1)
	gt.append(burial_frac_gt)
	approx.append(burial_frac_approx)

plt.scatter(gt, approx)
plt.xlabel('Ground Truth Burial Percentage')
plt.ylabel('Approximation with 0.5 - shift')